# Training Analysis for DRL Portfolio

This notebook analyzes the training process and hyperparameter sensitivity.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from data_acquisition import DataAcquisition
from portfolio_env import PortfolioEnv
from rl_agent import DRLAgent

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load Data and Create Environments

In [ ]:
# Load data
data_acq = DataAcquisition('config.yaml')
dataset = data_acq.fetch_full_dataset()

# Create environments
train_env = PortfolioEnv(
    prices=dataset['train']['prices'],
    returns=dataset['train']['returns']
)

val_env = PortfolioEnv(
    prices=dataset['val']['prices'],
    returns=dataset['val']['returns']
)

print(f"Training data: {len(dataset['train']['prices'])} days")
print(f"Validation data: {len(dataset['val']['prices'])} days")

## 2. Train Agent (Quick Test)

**Note:** For full training (500k timesteps), run the main.py script.

In [ ]:
# Create agent
agent = DRLAgent(train_env, algorithm='ppo')

print(f"Algorithm: {agent.algorithm_name.upper()}")
print(f"Policy: {agent.model.policy}")

In [ ]:
# Quick training test (10k steps instead of 500k)
print("Running quick training test (10k timesteps)...\n")

# Temporarily override config for quick test
agent.training_config['total_timesteps'] = 10000

# Train
agent.train(eval_env=val_env, save_path='models/test_model')

## 3. Evaluate Trained Agent

In [ ]:
# Evaluate on validation set
eval_results = agent.evaluate(val_env, n_episodes=5)

print("\nValidation Results:")
for metric, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")
    else:
        print(f"  {metric}: {value}")

## 4. Analyze TensorBoard Logs

**Note:** After full training, use TensorBoard to visualize:
- `tensorboard --logdir=logs/`

In [ ]:
# Placeholder for TensorBoard log analysis
print("""\nTo view training curves:

1. Run full training: python main.py --mode train
2. Start TensorBoard: tensorboard --logdir=logs/
3. Open browser: http://localhost:6006

You'll see:
- Episode reward mean (performance over time)
- Value loss (critic network learning)
- Policy loss (actor network learning)
- Entropy (exploration vs exploitation)
""")

## 5. Test Policy on Sample Episode

In [ ]:
# Run one episode with trained policy
obs, _ = val_env.reset()
done = False
step = 0
max_steps = 50

while not done and step < max_steps:
    action = agent.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = val_env.step(action)
    done = terminated or truncated
    step += 1

print(f"Episode completed in {step} steps")
print(f"Final portfolio value: ${val_env.portfolio_value:.4f}")

In [ ]:
# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Portfolio value
ax1.plot(val_env.portfolio_history, linewidth=2, color='blue')
ax1.set_title('Trained Policy - Portfolio Value', fontsize=14, fontweight='bold')
ax1.set_xlabel('Step', fontsize=12)
ax1.set_ylabel('Portfolio Value', fontsize=12)
ax1.grid(True, alpha=0.3)

# Weights evolution
weights_array = np.array(val_env.weights_history)
symbols = dataset['val']['prices'].columns

for i, symbol in enumerate(symbols):
    ax2.plot(weights_array[:, i], label=symbol, linewidth=2)

ax2.set_title('Trained Policy - Weight Evolution', fontsize=14, fontweight='bold')
ax2.set_xlabel('Step', fontsize=12)
ax2.set_ylabel('Weight', fontsize=12)
ax2.legend(fontsize=10)
ax2.axhline(y=0.4, color='red', linestyle='--', alpha=0.5, label='Max Weight')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Hyperparameter Sensitivity (Conceptual)

**Note:** For full hyperparameter optimization, use Optuna integration in main.py

In [ ]:
# Placeholder for hyperparameter sensitivity analysis
print("""\nHyperparameter Optimization:

The config.yaml includes Optuna settings for optimizing:
- learning_rate: [0.0001, 0.001]
- gamma: [0.95, 0.999]
- ent_coef: [0.0, 0.1]
- volatility_penalty: [0.0, 1.0]

To run full hyperparameter search:
1. Implement Optuna study in main.py
2. Run 50 trials (parallelized)
3. Select best by validation Sharpe ratio

Expected impact:
- Learning rate: Trade-off between speed and stability
- Gamma: Long-term vs short-term focus
- Entropy coefficient: Exploration vs exploitation
- Volatility penalty: Risk-return preference
""")

## 7. Learning Progress Visualization

In [ ]:
# Simulate learning progress (placeholder)
# In practice, this would come from TensorBoard logs

timesteps = np.arange(0, 500000, 10000)
reward_mean = -0.01 + 0.015 * (1 - np.exp(-timesteps / 100000))
reward_std = 0.02 * np.exp(-timesteps / 200000)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(timesteps, reward_mean, linewidth=2, label='Mean Reward')
ax.fill_between(
    timesteps,
    reward_mean - reward_std,
    reward_mean + reward_std,
    alpha=0.3,
    label='±1 Std Dev'
)
ax.set_title('Learning Progress (Simulated)', fontsize=14, fontweight='bold')
ax.set_xlabel('Timesteps', fontsize=12)
ax.set_ylabel('Episode Reward', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Note: This is a simulated learning curve. Run full training to see actual progress.")